---
title: Python and R and maximum likelihood estimate
toc: true
toc_sticky: true
---

In [1]:
# Load packages for coding examples
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Enables R magic
%load_ext rpy2.ipython

# Setup
Test of running Python and R in the same Jupyter notebook

In [3]:
df = pd.DataFrame({
    'cups_of_coffee': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    'productivity': [2, 5, 6, 8, 9, 8, 0, 1, 0, -1]
})

In [4]:
df

,cups_of_coffee,productivity
0,0,2
1,1,5
2,2,6
3,3,8
4,4,9
5,5,8
6,6,0
7,7,1
8,8,0
9,9,-1


## Testing use of R in notebook

In [5]:
%%R -i df -w 2 -h 2 --units in -r 200
# import df from global environment
# make default figure size 5 by 5 inches with 200 dpi resolution

#install.packages("ggplot2", repos='http://cran.us.r-project.org', quiet=TRUE)
# install.packages(nlmeU)
library(ggplot2)
library(dplyr)
library(tidyr)
library(nlme)
# ggplot(df, aes(x=cups_of_coffee, y=productivity)) + geom_line()


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘nlme’


R[write to console]: The following object is masked from ‘package:dplyr’:

    collapse




In [6]:
%%R

sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Mojave 10.14.6

Matrix products: default
BLAS:   /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib
LAPACK: /Users/lacar/anaconda/envs/insight/lib/R/lib/libRblas.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] nlme_3.1-148  tidyr_1.1.0   dplyr_1.0.0   ggplot2_3.3.2

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.5       lattice_0.20-41  withr_2.2.0      crayon_1.3.4    
 [5] grid_3.6.1       R6_2.4.1         lifecycle_0.2.0  gtable_0.3.0    
 [9] magrittr_1.5     scales_1.1.1     pillar_1.4.4     rlang_0.4.6     
[13] generics_0.0.2   vctrs_0.3.1      ellipsis_0.3.1   glue_1.4.1      
[17] purrr_0.3.4      munsell_0.5.0    compiler_3.6.1   pkgconfig_

## Importing the ARMD dataset

In [7]:
%%R

dataDir <- file.path("/Users/lacar/Documents/Data_science/Jupyter_notebooks/_my_DS_notes/analysis_and_machine_learning/LMM_data")
fp <- file.path(dataDir, "armd240.data.csv")
armd240.data <- read.csv(fp, header = TRUE)
dim(armd240.data)

[1] 240   9


In [14]:
%%R
head(armd240.data)

  subject treat lesion line0 visual0 visual4 visual12 visual24 visual52
1       1     2      3    12      59      55       45       NA       NA
2       2     2      1    13      65      70       65       65       55
3       3     1      4     8      40      40       37       17       NA
4       4     1      2    13      67      64       64       64       68
5       5     2      1    14      70      NA       NA       NA       NA
6       6     2      3    12      59      53       52       53       42


### Dataframe to Python

Allowing R dataframe to be used in pandas. From [this link](https://pandas.pydata.org/pandas-docs/version/0.22/r_interface.html).

In [9]:
from rpy2.robjects import r, pandas2ri

In [10]:
#%%R

pandas2ri.activate()
#df_pandas_armd = 
r.data("armd240.data")

'armd240.data'


In [11]:
r["armd240.data"].head()

,subject,treat,lesion,line0,visual0,visual4,visual12,visual24,visual52
1,1,2,3,12,59,55,45,-2147483648,-2147483648
2,2,2,1,13,65,70,65,65,55
3,3,1,4,8,40,40,37,17,-2147483648
4,4,1,2,13,67,64,64,64,68
5,5,2,1,14,70,-2147483648,-2147483648,-2147483648,-2147483648


In [12]:
type(r["armd240.data"])

pandas.core.frame.DataFrame

In [16]:
r["armd240.data"].loc[:, "subject"].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

### Loading data in the wide format (R)



In [15]:
%%R

armd.wide <- armd240.data %>% 
                spread(subject)

head(armd.wide)

R[write to console]: Error: object '' not found
Run `rlang::last_error()` to see where the error occurred.




Error: object '' not found
Run `rlang::last_error()` to see where the error occurred.


In [18]:
%%R

#rlang::last_error()

rlang::last_traceIO

R[write to console]: Error: 'last_traceIO' is not an exported object from 'namespace:rlang'




Error: 'last_traceIO' is not an exported object from 'namespace:rlang'


In [32]:
%%R

data(armd.wide, package = "nlmeU")
str(armd.wide)

R[write to console]: Error in find.package(package, lib.loc, verbose = verbose) : 
  there is no package called ‘nlmeU’




Error in find.package(package, lib.loc, verbose = verbose) : 
  there is no package called ‘nlmeU’
